# Fake News Classifier

### Data Reading and Interpretation

__Import Statements__

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
train_df = pd.read_csv("../data/raw/train.csv")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [4]:
train_df["author"].value_counts().value_counts()#.to_csv("temp.csv")

1      2615
2       475
3       217
4       155
5       109
       ... 
72        1
60        1
65        1
63        1
243       1
Name: author, Length: 89, dtype: int64

In [5]:
train_df.query("author == 'Guest'")["label"].value_counts()

1    39
Name: label, dtype: int64

In [6]:
# train_df.query("author.notnull()")["label"].value_counts()
# train_df.query("author.notnull()")[train_df.query("author.notnull()")["author"].str.contains("^[A-Z ]+$")]["label"].value_counts()
train_df.query("text == ' '")["label"].value_counts()

1    75
Name: label, dtype: int64

In [7]:
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
train_df.describe(include='all')

,id,title,author,text,label
count,20800.000000,20242,18843,20761,20800.000000
unique,NaN,19803,4201,20386,NaN
top,NaN,Get Ready For Civil Unrest: Survey Finds That ...,Pam Key,,NaN
freq,NaN,5,243,75,NaN
mean,10399.500000,NaN,NaN,NaN,0.500625
std,6004.587135,NaN,NaN,NaN,0.500012
min,0.000000,NaN,NaN,NaN,0.000000
25%,5199.750000,NaN,NaN,NaN,0.000000
50%,10399.500000,NaN,NaN,NaN,1.000000
75%,15599.250000,NaN,NaN,NaN,1.000000


### Preprocessing

#### Author

In [9]:
## Others category if value_counts of an author is less than 5

less_frequent = train_df['author'].value_counts()[train_df['author'].value_counts() <= 5].index.tolist()
train_df['author'] = np.where(train_df['author'].isin(less_frequent), 'Other', train_df['author'])

In [10]:
# Change Author = Null to Author = "Unknown"

unknown_authors_ids = train_df.query("author.isnull()")["id"]
train_df[updated_author_column_name] = np.where(~train_df['id'].isin(unknown_authors_ids), train_df['author'], 'Unknown')

In [11]:
# Create a feature "author_is_null"

def author_is_null(x):
    if x["author"] != x["author"]:
        return 0
    return 1

train_df["author_is_null"] = train_df.apply(lambda x: is_null_author(x), axis=1)

In [12]:
# Create if "is_multiple_authors"

def is_multiple_authors(data):

    data["is_multiple_authors"] = [
        1 if " and " in str(author) else 0 for author in data["author"]
    ]

    return data

In [13]:
train_df = is_multiple_authors(train_df)

In [14]:
train_df.query("is_multiple_authors == 1")["label"].value_counts()

0    1521
1      21
Name: label, dtype: int64

In [15]:
# Check if author name contains a domain suffix

def author_contains_domain(data):

    data["author_contains_domain"] = [
        1 if re.search(r"\.[a-zA-Z]{3}", str(author)) else 0 for author in train_df["author"]
    ]

    return data

In [16]:
train_df = author_contains_domain(train_df)

In [17]:
train_df.query("author_contains_domain == 1")["label"].value_counts()

1    521
Name: label, dtype: int64

#### Text

In [18]:
# Check if text is empty

def is_text_empty(data):

    data["is_text_empty"] = [
        1 if text == " " else 0 for text in train_df["text"]
    ]

    return data
# train_df.query("text == ' '")["label"].value_counts()

In [19]:
train_df = is_text_empty(train_df)

In [20]:
train_df.query("is_text_empty == 1")["label"].value_counts()

1    75
Name: label, dtype: int64

#### Feature Engineering

### Modelling

### Prediction and Results